In [2]:
from deap import gp
import operator
import math
import numpy as np

## Primitive set

In [3]:
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(math.cos, 1)

pset.renameArguments(ARG0='x')

## Helpers

In [82]:
def generate_random_tree(pset):
    expr = gp.genHalfAndHalf(pset, min_=1, max_=3)
    tree = gp.PrimitiveTree(expr)
    return tree

def build_primitives_terminals_dict(pset):
    prims = dict()
    prims_funcs = list(pset.primitives.values())[0]
    prims_names = [p.name for p in prims_funcs]
    prims.update(zip(prims_names, prims_funcs))

    # for arguments, add key = value = name to the dict
    for arg in pset.arguments:
        prims[str(arg)] = arg

    return prims

def tree_to_nodes_matrix(tree: gp.PrimitiveTree, pset: gp.PrimitiveSet, prims_names: list, n_nodes=0):
    n_prims = pset.prims_count + len(pset.arguments)
    if n_nodes == 0:
        n_nodes = len(tree)
    m = np.zeros((n_nodes, n_prims))

    for i, prim in enumerate(tree):
        prim_name = prim.name.replace('ARG0', 'x')
        prim_idx = prims_names.index(prim_name)
        m[i, prim_idx] = 1.
    
    return m

def eval_fitness(tree, pset, points):
    func = gp.compile(tree, pset)

    sqerrors = ((func(x) - x**2)**2 for x in points)
    return math.fsum(sqerrors) / len(points)

def generate_dataset(n_samples, max_nodes, pset, points, prims_names):
    n_prims = pset.prims_count + len(pset.arguments)
    
    X = np.zeros((n_samples, max_nodes*n_prims))
    y = np.zeros((n_samples, 1))
    
    for i in range(n_samples):
        tree = generate_random_tree(pset)
        m = tree_to_nodes_matrix(tree, pset, prims_names, max_nodes).ravel()
        X[i,:] = m
        fit = eval_fitness(tree, pset, points)
        if math.isnan(fit) or math.isinf(fit):
            fit = 1e5
        y[i,:] = fit

    return X, y
    
    

## Generation of datasets

In [83]:
tree = generate_random_tree(pset)
print(tree)
prims = build_primitives_terminals_dict(pset)
tree_to_nodes_matrix(tree, pset, list(prims.keys()))

protectedDiv(x, x)


array([[0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]])

In [84]:
points = np.arange(0.,1.1,0.1)
print(points)
eval_fitness(tree, pset, points)

[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


/tmp/ipykernel_34140/2793393697.py:3: RuntimeWarning: invalid value encountered in scalar divide
  return left / right


nan

In [85]:
from torch.utils.data import Dataset, DataLoader, random_split

In [91]:
X, y = generate_dataset(10, 15, pset, points, list(prims.keys()))
frac = 0.5
X_train, X_valid = random_split(X, [frac, 1-frac])
y_train, y_valid = random_split(y, [frac, 1-frac])

/tmp/ipykernel_34140/2793393697.py:3: RuntimeWarning: invalid value encountered in scalar divide
  return left / right


In [87]:
class CustomDataset(Dataset):
    def __init__(self, X, y, transform=None, target_transform=None):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return X[idx,:], y[idx, 0]

train_dataset = CustomDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset)

In [196]:
creator.create("Individual", gp.PrimitiveTree)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)

expr = toolbox.individual()
nodes, edges, labels = gp.graph(expr)

NameError: name 'creator' is not defined